In [1]:
import pandas as pd
import numpy as np
import os

os.listdir('./data')

['TCGA_binary.csv',
 'TCGA_data.csv',
 'urine_binary.csv',
 'urine_k15_0419.csv',
 'urine_K15_0420.csv',
 'urine_k15_0423.csv']

In [2]:
TCGA = pd.read_csv('./data/TCGA_data.csv')
urine = pd.read_csv('./data/urine_k15_0423.csv')
#urine = urine.drop(['Unnamed: 0','X'], axis = 1)
#urine.to_csv('./data/urine_K15_0420.csv', index = False)

In [3]:
urine.head()

,Unnamed: 0,X,cyto.1p36.33,cyto.1p36.32,cyto.1p36.31,cyto.1p36.23,cyto.1p36.22,cyto.1p36.21,cyto.1p36.13,cyto.1p36.12,...,cyto.9p21.1,cyto.9p13.3,cyto.9p13.2,cyto.9p13.1,cyto.9p12,cyto.9p11.2,cyto.9p11.1,cyto.9q11,cyto.9q12,y
0,1,1,-0.042049,-0.016378,-0.153316,-0.056765,-0.137091,0.178450,0.012607,-0.086320,...,-0.231586,-0.254872,-0.270547,-0.130938,0,0,0,0,0,BLCA_15k
1,2,2,0.254768,0.280497,0.113957,0.233515,0.315840,0.257673,0.254221,0.268129,...,0.258530,0.163861,0.050568,0.087438,0,0,0,0,0,BLCA_15k
2,3,3,0.008280,-0.046070,-0.019274,-0.038621,-0.013342,-0.034047,-0.054234,-0.038569,...,-0.004863,0.016083,-0.029978,0.067875,0,0,0,0,0,BLCA_15k
3,4,4,0.027988,0.009892,-0.039769,-0.036227,-0.055723,-0.064246,-0.051865,-0.062907,...,0.005607,-0.048461,-0.032935,0.043250,0,0,0,0,0,BLCA_15k
4,5,5,-0.060927,-0.092065,-0.078248,0.335235,0.131411,0.014468,-0.044209,0.038249,...,-0.022414,0.064189,0.128281,-0.113687,0,0,0,0,0,BLCA_15k


In [4]:
TCGA.y.value_counts()

Normal    1706
KIRC       528
PRAD       492
BLCA       408
KIRP       288
KICH        66
Name: y, dtype: int64

In [5]:
print('TCGA shape', TCGA.shape)
print('urine shape', urine.shape)

TCGA shape (3488, 787)
urine shape (116, 789)


In [6]:
TCGA = TCGA[(TCGA.y == 'BLCA') | (TCGA.y == 'Normal')]
TCGA = TCGA.replace(['BLCA','Normal'],[ 1,0 ])
TCGA = TCGA.reset_index(drop = True)

In [7]:
TCGA.shape

(2114, 787)

In [8]:
TCGA.y.value_counts()

0    1706
1     408
Name: y, dtype: int64

In [9]:
#NA 없음
TCGA.isnull().sum().sum()

0

In [10]:
urine.y.value_counts()

BLCA_15k    42
NL_15k      28
RCC_100k    23
PRAD_15k    23
Name: y, dtype: int64

In [11]:
urine = urine[(urine.y == 'BLCA_15k') | (urine.y == 'NL_15k')]
urine = urine.replace(['BLCA_15k','NL_15k'],[ 1,0 ])
urine = urine.reset_index(drop = True)

In [12]:
urine = urine.drop(['Unnamed: 0','X'], axis = 1)

In [13]:
urine.y.value_counts()

1    42
0    28
Name: y, dtype: int64

In [20]:
#TCGA.to_csv('./data/TCGA_binary.csv', index = False)
#urine.to_csv('./data/urine_binary.csv', index = False)

# scaling

In [14]:
X = TCGA.drop('y', axis = 1)
y = TCGA['y']

urine_X = urine.drop('y', axis = 1)
urine_y = urine['y']

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

standardsc = StandardScaler()
mms = MinMaxScaler()
mas = MaxAbsScaler()

In [16]:
X_standard = standardsc.fit_transform(X)
X_mms = mms.fit_transform(X)
X_mas = mas.fit_transform(X)

urine_X_standard = standardsc.fit_transform(urine_X)
urine_X_mms = mms.fit_transform(urine_X)
urine_X_mas = mas.fit_transform(urine_X)

C:\Users\wjssm\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\wjssm\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\wjssm\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

def get_eval(y_test, pred) :
    
    print('accuracy : {0:.4f} precision : {1:.4f} recall : {2:.4f} F1 : {3:.4f} AUC : {4:.4f}'.format(
                                                        accuracy_score(y_test, pred), 
                                                        precision_score(y_test, pred),
                                                        recall_score(y_test, pred),
                                                        f1_score(y_test,pred),
                                                        roc_auc_score(y_test, pred)
                                                                      ))
    
    
    
    #print('accuracy : {0:.4f}'.format(accuracy_score(y_test,pred)), 'roc_auc_score : ', roc_auc_score(y_test,pred))
    #print(classification_report(y_test, pred))
    print(confusion_matrix(y_test, pred))

In [33]:
from sklearn.preprocessing import Binarizer

def get_eval_by_threshold(y_test, pred_prba_c1, thresholds) :
    
    for custom_threshold in thresholds :
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_prba_c1)
        custom_predict = binarizer.transform(pred_prba_c1)
        
        print('threshold ; ', custom_threshold)
        get_eval(y_test, custom_predict)
        print('\n')

In [18]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5, random_state=0, shuffle = True)

# logistic regression

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

In [20]:
def model_pipeline(model) :
    pipeline1 = Pipeline([('StandardScaler', standardsc), ('Logistic', model)])
    pipeline2 = Pipeline([('MinMaxScaler', mms), ('Logistic', model)])
    pipeline3 = Pipeline([('MaxAbsScaler', mas), ('Logistic', model)])

    cv_acc = []
    pip1_acc = []
    pip2_acc = []
    pip3_acc = []

    for i, (tr_ind, te_ind) in enumerate(skf.split(X,y)) :

        X_train, X_test = X.iloc[tr_ind], X.iloc[te_ind]
        y_train, y_test = y[tr_ind], y[te_ind]

        model.fit(X_train, y_train)
        pipeline1.fit(X_train, y_train)
        pipeline2.fit(X_train, y_train)
        pipeline3.fit(X_train, y_train)

        cv_acc.append(accuracy_score(y[te_ind], model.predict(X_test)))
        pip1_acc.append(accuracy_score(y[te_ind], pipeline1.predict(X_test)))
        pip2_acc.append(accuracy_score(y[te_ind], pipeline2.predict(X_test)))
        pip3_acc.append(accuracy_score(y[te_ind], pipeline3.predict(X_test)))

        print('{0} 번째 accuracy non_scale : {1:.4f}% StandardScale : {2:.4f}% MinMax : {3:.4f}% MaxAbs : {4:.4f}%'.format(i,cv_acc[i],pip1_acc[i],pip2_acc[i],pip3_acc[i]))

    print('\n mean accuracy non_scale : {0:.4f}% StandardScale : {1:.4f}% MinMax : {2:.4f}% MaxAbs : {3:.4f}%'.format(np.mean(cv_acc),np.mean(pip1_acc),np.mean(pip2_acc),np.mean(pip3_acc)))

In [21]:
#predict urine data using pipeline

def pipeline_urine(model) :
    
    pipeline1 = Pipeline([('StandardScaler', standardsc), (model.__class__.__name__, model)])
    pipeline2 = Pipeline([('MinMaxScaler', mms), (model.__class__.__name__, model)])
    pipeline3 = Pipeline([('MaxAbsScaler', mas), (model.__class__.__name__, model)])
    
    model.fit(X,y)
    pipeline1.fit(X,y)
    pipeline2.fit(X,y)
    pipeline3.fit(X,y)
    
    print('###non-scaled###')
    get_eval(urine_y, model.predict(urine_X))
    print('###StandardScaler###')
    get_eval(urine_y, pipeline1.predict(urine_X))
    print('###MinMaxScaler###')
    get_eval(urine_y, pipeline2.predict(urine_X))
    print('###MaxAbsScaler###')
    get_eval(urine_y, pipeline3.predict(urine_X))

In [22]:
model_pipeline(lr)

0 번째 accuracy non_scale : 0.9788% StandardScale : 0.9764% MinMax : 0.8137% MaxAbs : 0.9741%
1 번째 accuracy non_scale : 0.9905% StandardScale : 0.9882% MinMax : 0.8132% MaxAbs : 0.9905%
2 번째 accuracy non_scale : 0.9787% StandardScale : 0.9787% MinMax : 0.8132% MaxAbs : 0.9716%
3 번째 accuracy non_scale : 0.9858% StandardScale : 0.9858% MinMax : 0.8128% MaxAbs : 0.9834%
4 번째 accuracy non_scale : 0.9763% StandardScale : 0.9787% MinMax : 0.8128% MaxAbs : 0.9739%

 mean accuracy non_scale : 0.9820% StandardScale : 0.9816% MinMax : 0.8131% MaxAbs : 0.9787%


In [117]:
pipeline_urine(lr)

###non-scaled###
accuracy : 0.4571 precision : 0.8333 recall : 0.1190 F1 : 0.2083 AUC : 0.5417
[[27  1]
 [37  5]]
###StandardScaler###
accuracy : 0.4714 precision : 0.8571 recall : 0.1429 F1 : 0.2449 AUC : 0.5536
[[27  1]
 [36  6]]
###MinMaxScaler###
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]
###MaxAbsScaler###
accuracy : 0.4857 precision : 0.8000 recall : 0.1905 F1 : 0.3077 AUC : 0.5595
[[26  2]
 [34  8]]


In [118]:
lr.fit(X_mas,y)
thresholds = [0.3,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, lr.predict_proba(urine_X_mas)[:,1].reshape(-1,1), thresholds)

threshold ;  0.3
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]


threshold ;  0.4
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]


threshold ;  0.5
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]


threshold ;  0.6
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]


threshold ;  0.7
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]




# randomforest

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [37]:
rf = RandomForestClassifier(n_estimators=20, random_state=0)

In [38]:
model_pipeline(rf)

0 번째 accuracy non_scale : 0.9858% StandardScale : 0.1981% MinMax : 0.1934% MaxAbs : 0.9835%
1 번째 accuracy non_scale : 0.9905% StandardScale : 0.9574% MinMax : 0.1939% MaxAbs : 0.9905%
2 번째 accuracy non_scale : 0.9835% StandardScale : 0.9291% MinMax : 0.1939% MaxAbs : 0.9835%
3 번째 accuracy non_scale : 0.9858% StandardScale : 0.9739% MinMax : 0.1919% MaxAbs : 0.9858%
4 번째 accuracy non_scale : 0.9763% StandardScale : 0.8389% MinMax : 0.1919% MaxAbs : 0.9787%

 mean accuracy non_scale : 0.9844% StandardScale : 0.7795% MinMax : 0.1930% MaxAbs : 0.9844%


In [39]:
pipeline_urine(rf)

###non-scaled###
accuracy : 0.6143 precision : 0.6087 recall : 1.0000 F1 : 0.7568 AUC : 0.5179
[[ 1 27]
 [ 0 42]]
###StandardScaler###
accuracy : 0.6429 precision : 0.6308 recall : 0.9762 F1 : 0.7664 AUC : 0.5595
[[ 4 24]
 [ 1 41]]
###MinMaxScaler###
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]
###MaxAbsScaler###
accuracy : 0.6000 precision : 0.6061 recall : 0.9524 F1 : 0.7407 AUC : 0.5119
[[ 2 26]
 [ 2 40]]


In [41]:
rf.fit(X_standard,y)
thresholds = [0.3,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, rf.predict_proba(urine_X_standard)[:,1].reshape(-1,1), thresholds)

threshold ;  0.3
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.4
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.5
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.6
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.7
accuracy : 0.5714 precision : 0.5882 recall : 0.9524 F1 : 0.7273 AUC : 0.4762
[[ 0 28]
 [ 2 40]]




In [120]:
rf.fit(X,y)
thresholds = [0.3,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, rf.predict_proba(urine_X)[:,1].reshape(-1,1), thresholds)

threshold ;  0.3
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.4
accuracy : 0.6143 precision : 0.6087 recall : 1.0000 F1 : 0.7568 AUC : 0.5179
[[ 1 27]
 [ 0 42]]


threshold ;  0.5
accuracy : 0.5857 precision : 0.6000 recall : 0.9286 F1 : 0.7290 AUC : 0.5000
[[ 2 26]
 [ 3 39]]


threshold ;  0.6
accuracy : 0.6000 precision : 0.6167 recall : 0.8810 F1 : 0.7255 AUC : 0.5298
[[ 5 23]
 [ 5 37]]


threshold ;  0.7
accuracy : 0.6143 precision : 0.6415 recall : 0.8095 F1 : 0.7158 AUC : 0.5655
[[ 9 19]
 [ 8 34]]




# adaboost

In [42]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()

In [61]:
model_pipeline(ada)

0 번째 accuracy non_scale : 0.9788% StandardScale : 0.8703% MinMax : 0.1934% MaxAbs : 0.9858%
1 번째 accuracy non_scale : 0.9905% StandardScale : 0.9811% MinMax : 0.1939% MaxAbs : 0.9882%
2 번째 accuracy non_scale : 0.9693% StandardScale : 0.9740% MinMax : 0.1939% MaxAbs : 0.9716%
3 번째 accuracy non_scale : 0.9787% StandardScale : 0.2133% MinMax : 0.1919% MaxAbs : 0.9787%
4 번째 accuracy non_scale : 0.9882% StandardScale : 0.9739% MinMax : 0.1919% MaxAbs : 0.9834%

 mean accuracy non_scale : 0.9811% StandardScale : 0.8025% MinMax : 0.1930% MaxAbs : 0.9815%


In [121]:
pipeline_urine(ada)

###non-scaled###
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]
###StandardScaler###
accuracy : 0.6714 precision : 0.6610 recall : 0.9286 F1 : 0.7723 AUC : 0.6071
[[ 8 20]
 [ 3 39]]
###MinMaxScaler###
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]
###MaxAbsScaler###
accuracy : 0.6714 precision : 0.7021 recall : 0.7857 F1 : 0.7416 AUC : 0.6429
[[14 14]
 [ 9 33]]


In [43]:
#best
ada.fit(X_mas,y)
thresholds = [0.4,0.45,0.5,0.55,0.6]
get_eval_by_threshold(urine_y, ada.predict_proba(urine_X_mas)[:,1].reshape(-1,1), thresholds)

threshold ;  0.4
accuracy : 0.5857 precision : 0.5942 recall : 0.9762 F1 : 0.7387 AUC : 0.4881
[[ 0 28]
 [ 1 41]]


threshold ;  0.45
accuracy : 0.7000 precision : 0.7333 recall : 0.7857 F1 : 0.7586 AUC : 0.6786
[[16 12]
 [ 9 33]]


threshold ;  0.5
accuracy : 0.6429 precision : 0.8696 recall : 0.4762 F1 : 0.6154 AUC : 0.6845
[[25  3]
 [22 20]]


threshold ;  0.55
accuracy : 0.5429 precision : 0.8125 recall : 0.3095 F1 : 0.4483 AUC : 0.6012
[[25  3]
 [29 13]]


threshold ;  0.6
accuracy : 0.4286 precision : 0.6667 recall : 0.0952 F1 : 0.1667 AUC : 0.5119
[[26  2]
 [38  4]]




# decision tree

In [45]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

In [189]:
model_pipeline(tree)

0 번째 accuracy non_scale : 0.9670% StandardScale : 0.1580% MinMax : 0.1934% MaxAbs : 0.9646%
1 번째 accuracy non_scale : 0.9551% StandardScale : 0.9409% MinMax : 0.1939% MaxAbs : 0.9740%
2 번째 accuracy non_scale : 0.9456% StandardScale : 0.1939% MinMax : 0.1939% MaxAbs : 0.9811%
3 번째 accuracy non_scale : 0.9123% StandardScale : 0.1635% MinMax : 0.1919% MaxAbs : 0.9645%
4 번째 accuracy non_scale : 0.9431% StandardScale : 0.2014% MinMax : 0.1919% MaxAbs : 0.9645%

 mean accuracy non_scale : 0.9446% StandardScale : 0.3315% MinMax : 0.1930% MaxAbs : 0.9697%


In [46]:
pipeline_urine(tree)

###non-scaled###
accuracy : 0.5571 precision : 0.5965 recall : 0.8095 F1 : 0.6869 AUC : 0.4940
[[ 5 23]
 [ 8 34]]
###StandardScaler###
accuracy : 0.5857 precision : 0.6140 recall : 0.8333 F1 : 0.7071 AUC : 0.5238
[[ 6 22]
 [ 7 35]]
###MinMaxScaler###
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]
###MaxAbsScaler###
accuracy : 0.5714 precision : 0.6111 recall : 0.7857 F1 : 0.6875 AUC : 0.5179
[[ 7 21]
 [ 9 33]]


In [48]:
tree.fit(X_mas,y)
thresholds = [0.4,0.45,0.5,0.55,0.6]
get_eval_by_threshold(urine_y, tree.predict_proba(urine_X_mas)[:,1].reshape(-1,1), thresholds)

threshold ;  0.4
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]


threshold ;  0.45
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]


threshold ;  0.5
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]


threshold ;  0.55
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]


threshold ;  0.6
accuracy : 0.6286 precision : 0.6481 recall : 0.8333 F1 : 0.7292 AUC : 0.5774
[[ 9 19]
 [ 7 35]]




# GradientBoostingClassifier

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()

In [192]:
model_pipeline(gb)

0 번째 accuracy non_scale : 0.9835% StandardScale : 0.9741% MinMax : 0.1934% MaxAbs : 0.9858%
1 번째 accuracy non_scale : 0.9835% StandardScale : 0.9551% MinMax : 0.1939% MaxAbs : 0.9882%
2 번째 accuracy non_scale : 0.9764% StandardScale : 0.9787% MinMax : 0.1939% MaxAbs : 0.9764%
3 번째 accuracy non_scale : 0.9810% StandardScale : 0.9834% MinMax : 0.1919% MaxAbs : 0.9858%
4 번째 accuracy non_scale : 0.9787% StandardScale : 0.9716% MinMax : 0.1919% MaxAbs : 0.9858%

 mean accuracy non_scale : 0.9806% StandardScale : 0.9726% MinMax : 0.1930% MaxAbs : 0.9844%


In [51]:
pipeline_urine(gb)

###non-scaled###
accuracy : 0.6143 precision : 0.6119 recall : 0.9762 F1 : 0.7523 AUC : 0.5238
[[ 2 26]
 [ 1 41]]
###StandardScaler###
accuracy : 0.6714 precision : 0.6667 recall : 0.9048 F1 : 0.7677 AUC : 0.6131
[[ 9 19]
 [ 4 38]]
###MinMaxScaler###
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]
###MaxAbsScaler###
accuracy : 0.6286 precision : 0.6250 recall : 0.9524 F1 : 0.7547 AUC : 0.5476
[[ 4 24]
 [ 2 40]]


In [52]:
gb.fit(X,y)
thresholds = [0.35,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, gb.predict_proba(urine_X)[:,1].reshape(-1,1), thresholds)

threshold ;  0.35
accuracy : 0.6143 precision : 0.6154 recall : 0.9524 F1 : 0.7477 AUC : 0.5298
[[ 3 25]
 [ 2 40]]


threshold ;  0.4
accuracy : 0.6286 precision : 0.6250 recall : 0.9524 F1 : 0.7547 AUC : 0.5476
[[ 4 24]
 [ 2 40]]


threshold ;  0.5
accuracy : 0.6286 precision : 0.6250 recall : 0.9524 F1 : 0.7547 AUC : 0.5476
[[ 4 24]
 [ 2 40]]


threshold ;  0.6
accuracy : 0.6286 precision : 0.6250 recall : 0.9524 F1 : 0.7547 AUC : 0.5476
[[ 4 24]
 [ 2 40]]


threshold ;  0.7
accuracy : 0.6429 precision : 0.6349 recall : 0.9524 F1 : 0.7619 AUC : 0.5655
[[ 5 23]
 [ 2 40]]




In [54]:
gb.fit(X_mas,y)
thresholds = [0.35,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, gb.predict_proba(urine_X_mas)[:,1].reshape(-1,1), thresholds)

threshold ;  0.35
accuracy : 0.6429 precision : 0.6889 recall : 0.7381 F1 : 0.7126 AUC : 0.6190
[[14 14]
 [11 31]]


threshold ;  0.4
accuracy : 0.6714 precision : 0.7209 recall : 0.7381 F1 : 0.7294 AUC : 0.6548
[[16 12]
 [11 31]]


threshold ;  0.5
accuracy : 0.7000 precision : 0.7692 recall : 0.7143 F1 : 0.7407 AUC : 0.6964
[[19  9]
 [12 30]]


threshold ;  0.6
accuracy : 0.6857 precision : 0.7941 recall : 0.6429 F1 : 0.7105 AUC : 0.6964
[[21  7]
 [15 27]]


threshold ;  0.7
accuracy : 0.6429 precision : 0.7931 recall : 0.5476 F1 : 0.6479 AUC : 0.6667
[[22  6]
 [19 23]]




# SVM

In [59]:
from sklearn.svm import SVC
svm = SVC(probability=True)

In [199]:
model_pipeline(svm)

0 번째 accuracy non_scale : 0.9693% StandardScale : 0.9858% MinMax : 0.8986% MaxAbs : 0.9599%
1 번째 accuracy non_scale : 0.9764% StandardScale : 0.9882% MinMax : 0.8889% MaxAbs : 0.9669%
2 번째 accuracy non_scale : 0.9551% StandardScale : 0.9787% MinMax : 0.8629% MaxAbs : 0.9504%
3 번째 accuracy non_scale : 0.9668% StandardScale : 0.9882% MinMax : 0.8815% MaxAbs : 0.9597%
4 번째 accuracy non_scale : 0.9692% StandardScale : 0.9810% MinMax : 0.9005% MaxAbs : 0.9645%

 mean accuracy non_scale : 0.9674% StandardScale : 0.9844% MinMax : 0.8865% MaxAbs : 0.9603%


In [60]:
pipeline_urine(svm)

###non-scaled###
accuracy : 0.5571 precision : 0.8235 recall : 0.3333 F1 : 0.4746 AUC : 0.6131
[[25  3]
 [28 14]]
###StandardScaler###
accuracy : 0.6429 precision : 0.6441 recall : 0.9048 F1 : 0.7525 AUC : 0.5774
[[ 7 21]
 [ 4 38]]
###MinMaxScaler###
accuracy : 0.5286 precision : 0.8000 recall : 0.2857 F1 : 0.4211 AUC : 0.5893
[[25  3]
 [30 12]]
###MaxAbsScaler###
accuracy : 0.5143 precision : 0.7857 recall : 0.2619 F1 : 0.3929 AUC : 0.5774
[[25  3]
 [31 11]]


In [64]:
svm.fit(X_standard,y)
thresholds = [0.35,0.4,0.45,0.5,0.55]
get_eval_by_threshold(urine_y, svm.predict_proba(urine_X_standard)[:,1].reshape(-1,1), thresholds)

threshold ;  0.35
accuracy : 0.5857 precision : 0.7407 recall : 0.4762 F1 : 0.5797 AUC : 0.6131
[[21  7]
 [22 20]]


threshold ;  0.4
accuracy : 0.6143 precision : 0.8261 recall : 0.4524 F1 : 0.5846 AUC : 0.6548
[[24  4]
 [23 19]]


threshold ;  0.45
accuracy : 0.5571 precision : 0.8235 recall : 0.3333 F1 : 0.4746 AUC : 0.6131
[[25  3]
 [28 14]]


threshold ;  0.5
accuracy : 0.5571 precision : 0.8235 recall : 0.3333 F1 : 0.4746 AUC : 0.6131
[[25  3]
 [28 14]]


threshold ;  0.55
accuracy : 0.5143 precision : 0.7857 recall : 0.2619 F1 : 0.3929 AUC : 0.5774
[[25  3]
 [31 11]]




# ligthGBM

In [86]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()

In [74]:
model_pipeline(lgb)

0 번째 accuracy non_scale : 0.9858% StandardScale : 0.9788% MinMax : 0.1934% MaxAbs : 0.9835%
1 번째 accuracy non_scale : 0.9858% StandardScale : 0.9905% MinMax : 0.1939% MaxAbs : 0.9882%
2 번째 accuracy non_scale : 0.9835% StandardScale : 0.9764% MinMax : 0.1939% MaxAbs : 0.9787%
3 번째 accuracy non_scale : 0.9834% StandardScale : 0.9834% MinMax : 0.1919% MaxAbs : 0.9882%
4 번째 accuracy non_scale : 0.9810% StandardScale : 0.9858% MinMax : 0.1919% MaxAbs : 0.9858%

 mean accuracy non_scale : 0.9839% StandardScale : 0.9830% MinMax : 0.1930% MaxAbs : 0.9849%


In [87]:
pipeline_urine(lgb)

accuracy : 0.6286 precision : 0.6250 recall : 0.9524 F1 : 0.7547 AUC : 0.5476
[[ 4 24]
 [ 2 40]]
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]
accuracy : 0.6286 precision : 0.6212 recall : 0.9762 F1 : 0.7593 AUC : 0.5417
[[ 3 25]
 [ 1 41]]


## not scaled

In [48]:
lgb = LGBMClassifier(n_estimators=400, learning_rate=0.01)
evals = [(urine_X, urine_y)]

lgb.fit(X ,y, early_stopping_rounds = 100, eval_metric = 'logloss', eval_set = evals, verbose = 50)

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.735598	valid_0's binary_logloss: 0.735598
[100]	valid_0's binary_logloss: 0.701504	valid_0's binary_logloss: 0.701504
[150]	valid_0's binary_logloss: 0.735156	valid_0's binary_logloss: 0.735156
[200]	valid_0's binary_logloss: 0.788854	valid_0's binary_logloss: 0.788854
Early stopping, best iteration is:
[100]	valid_0's binary_logloss: 0.701504	valid_0's binary_logloss: 0.701504


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [49]:
pred = lgb.predict(urine_X)
get_eval(urine_y, pred)

accuracy : 0.5143 precision : 0.5870 recall : 0.6429 F1 : 0.6136 AUC : 0.4821
[[ 9 19]
 [15 27]]


In [51]:
thresholds = [0.5,0.55,0.6,0.65,0.7,0]
get_eval_by_threshold(urine_y, lgb.predict_proba(urine_X)[:,1].reshape(-1,1), thresholds)

threshold ;  0.5
accuracy : 0.5143 precision : 0.5870 recall : 0.6429 F1 : 0.6136 AUC : 0.4821
[[ 9 19]
 [15 27]]


threshold ;  0.55
accuracy : 0.5714 precision : 0.6579 recall : 0.5952 F1 : 0.6250 AUC : 0.5655
[[15 13]
 [17 25]]


threshold ;  0.6
accuracy : 0.5714 precision : 0.6667 recall : 0.5714 F1 : 0.6154 AUC : 0.5714
[[16 12]
 [18 24]]


threshold ;  0.65
accuracy : 0.5857 precision : 0.8421 recall : 0.3810 F1 : 0.5246 AUC : 0.6369
[[25  3]
 [26 16]]


threshold ;  0.7
accuracy : 0.5429 precision : 0.9167 recall : 0.2619 F1 : 0.4074 AUC : 0.6131
[[27  1]
 [31 11]]


threshold ;  0
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]




## standard

In [64]:
lgb = LGBMClassifier(n_estimators=400, learning_rate=0.01)
evals = [(urine_X_standard, urine_y)]

lgb.fit(X_standard ,y, early_stopping_rounds = 100, eval_metric = 'logloss', eval_set = evals, verbose = 50)

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.705152	valid_0's binary_logloss: 0.705152
[100]	valid_0's binary_logloss: 0.718558	valid_0's binary_logloss: 0.718558
[150]	valid_0's binary_logloss: 0.821201	valid_0's binary_logloss: 0.821201
Early stopping, best iteration is:
[68]	valid_0's binary_logloss: 0.692048	valid_0's binary_logloss: 0.692048


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [56]:
pred = lgb.predict(urine_X_standard)
get_eval(urine_y, pred) #똥

accuracy : 0.5429 precision : 0.5758 recall : 0.9048 F1 : 0.7037 AUC : 0.4524
[[ 0 28]
 [ 4 38]]


## minmax

In [65]:
lgb = LGBMClassifier(n_estimators=500, learning_rate=0.01)
evals = [(urine_X_mms, urine_y)]

lgb.fit(X_mms ,y, early_stopping_rounds = 100, eval_metric = 'logloss', eval_set = evals, verbose = 50)

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.689839	valid_0's binary_logloss: 0.689839
[100]	valid_0's binary_logloss: 0.695689	valid_0's binary_logloss: 0.695689
[150]	valid_0's binary_logloss: 0.800281	valid_0's binary_logloss: 0.800281
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.673253	valid_0's binary_logloss: 0.673253


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [66]:
thresholds = [0.3,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, lgb.predict_proba(urine_X_mms)[:,1].reshape(-1,1), thresholds)

threshold ;  0.3
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.4
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.5
accuracy : 0.6000 precision : 0.6000 recall : 1.0000 F1 : 0.7500 AUC : 0.5000
[[ 0 28]
 [ 0 42]]


threshold ;  0.6
accuracy : 0.5714 precision : 0.5938 recall : 0.9048 F1 : 0.7170 AUC : 0.4881
[[ 2 26]
 [ 4 38]]


threshold ;  0.7
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]




## maxabs

In [67]:
lgb = LGBMClassifier(n_estimators=500, learning_rate=0.01)
evals = [(urine_X_mas, urine_y)]

lgb.fit(X_mas ,y, early_stopping_rounds = 100, eval_metric = 'logloss', eval_set = evals, verbose = 50)

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.835623	valid_0's binary_logloss: 0.835623
[100]	valid_0's binary_logloss: 0.829883	valid_0's binary_logloss: 0.829883
[150]	valid_0's binary_logloss: 0.845839	valid_0's binary_logloss: 0.845839
Early stopping, best iteration is:
[77]	valid_0's binary_logloss: 0.82165	valid_0's binary_logloss: 0.82165


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.01, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [68]:
thresholds = [0.3,0.4,0.5,0.6,0.7]
get_eval_by_threshold(urine_y, lgb.predict_proba(urine_X_mas)[:,1].reshape(-1,1), thresholds)

threshold ;  0.3
accuracy : 0.5286 precision : 0.6098 recall : 0.5952 F1 : 0.6024 AUC : 0.5119
[[12 16]
 [17 25]]


threshold ;  0.4
accuracy : 0.5429 precision : 0.6667 recall : 0.4762 F1 : 0.5556 AUC : 0.5595
[[18 10]
 [22 20]]


threshold ;  0.5
accuracy : 0.5286 precision : 0.7368 recall : 0.3333 F1 : 0.4590 AUC : 0.5774
[[23  5]
 [28 14]]


threshold ;  0.6
accuracy : 0.4143 precision : 0.6667 recall : 0.0476 F1 : 0.0889 AUC : 0.5060
[[27  1]
 [40  2]]


threshold ;  0.7
accuracy : 0.4000 precision : 0.0000 recall : 0.0000 F1 : 0.0000 AUC : 0.5000
[[28  0]
 [42  0]]


